In [1]:
#test_ignore
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#TODO: fill nan and -9999 values
#TODO: include final snow model creation once finalized what format we want to use

In [3]:
import zarr
import xarray as xr
import pandas as pd
from numcodecs import Blosc
import os
import numpy as np
import re
import glob
import rioxarray
from dask.distributed import Client, LocalCluster
import dask
import matplotlib.pyplot as plt

Product code 

1025: Precipitation

1034: Snow water equivalent

1036: Snow depth

1038: Snow pack average temperature

1039: Blowing snow sublimation

1044: Snow melt

1050: Snow pack sublimation

In [5]:
cluster = LocalCluster(n_workers=12)
client = Client(cluster)
client
                  

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 24,Total memory: 127.95 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:3243,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 127.95 GiB
Comm: tcp://127.0.0.1:3336,Total threads: 2
Dashboard: http://127.0.0.1:3337/status,Memory: 10.66 GiB
Nanny: tcp://127.0.0.1:3252,


In [4]:
from concurrent.futures import ThreadPoolExecutor
dask.config.set(pool=ThreadPoolExecutor(24))

In [6]:
dask.config.set({'temporary-directory': 'E:\Temp'})

In [8]:
dask.config.get

<function dask.config.get(key: 'str', default: 'Any' = '__no_default__', config: 'dict' = {'temporary-directory': 'E:\\Temp', 'tokenize': {'ensure-deterministic': False}, 'dataframe': {'shuffle-compression': None, 'parquet': {'metadata-task-size-local': 512, 'metadata-task-size-remote': 16}}, 'array': {'svg': {'size': 120}, 'slicing': {'split-large-chunks': None}, 'chunk-size': '128MiB', 'rechunk-threshold': 4}, 'optimization': {'fuse': {'active': None, 'ave-width': 1, 'max-width': None, 'max-height': inf, 'max-depth-new-edges': None, 'subgraphs': None, 'rename-keys': True}}, 'distributed': {'version': 2, 'scheduler': {'allowed-failures': 3, 'bandwidth': 100000000, 'blocked-handlers': [], 'default-data-size': '1kiB', 'events-cleanup-delay': '1h', 'idle-timeout': None, 'transition-log-length': 100000, 'events-log-length': 100000, 'work-stealing': True, 'work-stealing-interval': '100ms', 'worker-ttl': '5 minutes', 'pickle': True, 'preload': [], 'preload-argv': [], 'unknown-task-duration'

In [ ]:
def create_file_lists(prefix, suffix, files):
    file_list = []
    file_dates = []
    result = [re.search(prefix + '(\d{8})' + suffix, f) for f in files]
    l = [x.group(1) for x in result if x is not None]
    l.sort()
    file_dates += l
    file_list += [path + prefix + x + suffix for x in l ]
    return file_dates, file_list

In [ ]:
dates = pd.date_range('2010-01-01', '2012-12-31', freq='M')
precip_file_list = []
snowprecip_file_list = []
swe_file_list = []
depth_file_list = []
temp_file_list = []
blowing_sublimation_file_list = []
snowmelt_file_list = []
snowpack_sublimation_file_list = []
file_dates = []
for d in dates:
    #one iteration per month
    path = d.strftime('//openmediavault/DataCold/SnowData/snodas/unmasked/%Y/%m_%b/')
    dir_files = os.listdir(path)
    
    tmp_dates, files = create_file_lists('zz_ssmv01025SlL00T0024TTNATS', '05DP001.dat.nc', dir_files)
    precip_file_list += files
    #assuming if the date exists it exists for all files; maybe bad assumption but the merge should fail later 
    #will update logic if necessary
    file_dates += tmp_dates
    print('precip dates' + str(len(tmp_dates)))
    
    tmp_dates, files = create_file_lists('zz_ssmv01025SlL01T0024TTNATS', '05DP001.dat.nc', dir_files)
    snowprecip_file_list += files
    print('snow precip dates' + str(len(tmp_dates)))
    
    tmp_dates, files = create_file_lists('zz_ssmv11034tS__T0001TTNATS', '05HP001.dat.nc', dir_files)
    swe_file_list += files
    print('swe dates' + str(len(tmp_dates)))
        
    tmp_dates, files = create_file_lists('zz_ssmv11036tS__T0001TTNATS', '05HP001.dat.nc', dir_files)
    depth_file_list += files
    print('depth dates' + str(len(tmp_dates)))
        
    tmp_dates, files = create_file_lists('zz_ssmv11038wS__A0024TTNATS', '05DP001.dat.nc', dir_files)
    temp_file_list += files
    print('temp dates' + str(len(tmp_dates)))
    
    tmp_dates, files = create_file_lists('zz_ssmv11039lL00T0024TTNATS', '05DP000.dat.nc', dir_files)
    blowing_sublimation_file_list += files
    print('blowing dates' + str(len(tmp_dates)))
                                              
    snowmelt_dates, files = create_file_lists('zz_ssmv11044bS__T0001TTNATS', '05HP000.dat.nc', dir_files)
    snowmelt_file_list += files
    #print('snowmelt dates' + str(len(dates)))
    
    #snowmelt sometimes comes in as 24 hours
    tmp_dates, files = create_file_lists('zz_ssmv11044bS__T0024TTNATS', '05DP000.dat.nc', dir_files)
    snowmelt_file_list += files
    snowmelt_file_list.sort()
    print('snowmelt dates' + str(len(snowmelt_dates + tmp_dates)))
         
    tmp_dates, files = create_file_lists('zz_ssmv11050lL00T0024TTNATS', '05DP000.dat.nc', dir_files)
    snowpack_sublimation_file_list += files
    print('snowpack dates' + str(len(tmp_dates)))

In [ ]:
%%time 
#TODO: need to gather actual timestamps from files and apply them to the dataset to account for missing data
dates = pd.date_range('2010-01-01', '2012-12-31', freq='M')
p = False
name_dict = {'Band1': 'NonSnowPrecip'}
ds_precip = xr.open_mfdataset(precip_file_list, combine='nested', concat_dim='time', parallel=p).drop_vars('crs').rename_vars(name_dict)
name_dict = {'Band1': 'SnowPrecip'}
ds_snowprecip = xr.open_mfdataset(snowprecip_file_list, combine='nested', concat_dim='time', parallel=p).drop_vars('crs').rename_vars(name_dict)
name_dict = {'Band1': 'SWE'}
ds_swe = xr.open_mfdataset(swe_file_list, combine='nested', concat_dim='time', parallel=p).drop_vars('crs').rename_vars(name_dict)
name_dict = {'Band1': 'SnowDepth'}
ds_depth = xr.open_mfdataset(depth_file_list, combine='nested', concat_dim='time', parallel=p).drop_vars('crs').rename_vars(name_dict)
name_dict = {'Band1': 'SnowpackAvgTemp'}
ds_temp = xr.open_mfdataset(temp_file_list, combine='nested', concat_dim='time', parallel=p).drop_vars('crs').rename_vars(name_dict)
name_dict = {'Band1': 'BlowingSnowSublimation'}
ds_blowing_sublimation = xr.open_mfdataset(blowing_sublimation_file_list, combine='nested', concat_dim='time', parallel=p).drop_vars('crs').rename_vars(name_dict)
name_dict = {'Band1': 'SnowMelt'}
ds_snowmelt = xr.open_mfdataset(snowmelt_file_list, combine='nested', concat_dim='time', parallel=p).drop_vars('crs').rename_vars(name_dict)
name_dict = {'Band1': 'SnowpackSublimation'}
ds_snowpack_sublimation = xr.open_mfdataset(snowpack_sublimation_file_list, combine='nested', concat_dim='time', parallel=p).drop_vars('crs').rename_vars(name_dict)


In [ ]:
snow_datasets = [ds_precip, ds_snowprecip, ds_swe, ds_depth, ds_temp, ds_blowing_sublimation, ds_snowmelt, ds_snowpack_sublimation]

In [ ]:
ds_depth.SnowDepth[-200,:,:].plot()

In [ ]:
ds_snowpack_sublimation.SnowpackSublimation[-200,:,:].plot()

In [ ]:
ds_snowpack_sublimation.SnowpackSublimation[31,:,:].plot()

In [ ]:
ds_snowpack_sublimation.SnowpackSublimation[57,:,:].plot()

In [ ]:
ds_snowpack_sublimation.SnowpackSublimation[400,:,:].plot()

In [ ]:
%%time 

ds = xr.merge(snow_datasets)

In [ ]:
ds['time'] = pd.to_datetime(pd.Series(file_dates)).values

In [ ]:
#check ds for nan values
tmpnull = ds['NonSnowPrecip'].isnull().values

In [ ]:
for i in range(0,len(ds['SnowpackSublimation'].time),100):
    ds['SnowpackSublimation'].isel(time=i).plot()
    plt.show()

In [ ]:
np.count_nonzero(tmpnull)

In [ ]:
ds

In [ ]:
enc = {x: {"compressor": Blosc(cname='zstd', clevel=3, shuffle=Blosc.BITSHUFFLE)} for x in ds}

In [ ]:
%%time

#2 months parallel=False
#CPU times: total: 3min 39s
#Wall time: 27min 13s
#Wired CPU times: total: 2min 35s
#Wall time: 5min 52s
ds.chunk({'time': -1, 'lat': 500, 'lon': 500}).to_zarr('//openmediavault/DataCold/SnowData/snodas/snodas.zarr', consolidated=True, encoding=enc)

In [ ]:
ds2 = xr.open_zarr('//openmediavault/DataCold/SnowData/snodas/snodas.zarr')

In [ ]:
ds2

In [ ]:
#now we'll ensure the ERA5 data is in the right format

In [ ]:
ds_era5 = xr.open_mfdataset('//openmediavault/DataCold/ERA5/201*/*/*.nc', combine='nested', concat_dim='time')

In [ ]:
ds_era5

# After merged each geotiff for ASTER elevation in qgis to the 10s of latitudes (larger is too large for a chunk) 
# read each on in here and align and downsample to the same lat/lon in snodas

In [ ]:
directory = '//openmediavault.local/DataCold/GIS/AsterElevation/'
geotiff_da = None
geotiff_da = xr.open_dataset(directory + 'NA_Snodas_region_large_chunks_N30toN39_aspect.tif', engine='rasterio', mask_and_scale=False)


In [ ]:
geotiff_da = geotiff_da.drop_vars(['band', 'spatial_ref'])

In [ ]:
filter_x = [x for x in ds2.lon.values if x >= np.min(geotiff_da.x.values) and x <= np.max(geotiff_da.x.values)]
filter_y = [x for x in ds2.lat.values if x >= np.min(geotiff_da.y.values) and x <= np.max(geotiff_da.y.values)]

In [ ]:
tmp = geotiff_da.sel(x = slice(np.min(filter_x), np.max(filter_x)), y = slice(np.max(filter_y), np.min(filter_y))).interp(x=filter_x, y=filter_y)

In [ ]:
tmp.astype(np.int16).to_netcdf('//openmediavault.local/DataCold/GIS/AsterElevation/NA_Aligned_To_Snodas_N30toN39_aspect.nc')

# Review them as a single dataset

In [ ]:
elevation_ds = xr.open_mfdataset('//openmediavault.local/DataCold/GIS/AsterElevation/NA_Aligned_To_Snodas_*.nc', combine='nested')

In [ ]:
elevation_ds

In [ ]:
elevation_ds.band_data.plot()

# Rechunk era5 to be better aligned with the timeseries (all times in one chunk for smaller lat/lon chunks)

In [ ]:
ds_era5

In [ ]:
ds_era5 = ds_era5.rename({'longitude': 'lon', 'latitude': 'lat'})

In [ ]:
ds_era5

In [ ]:
%%time 

t_resample = ds_era5.resample(time='1D')
era5_daily_mean = t_resample.mean()
era5_daily_min = t_resample.min()
era5_daily_max = t_resample.max()
era5_daily_sum = t_resample.sum()
era5_daily_mean = era5_daily_mean.rename_vars({var: var + '_mean' for var in era5_daily_mean.data_vars})
era5_daily_min = era5_daily_min.rename_vars({var: var + '_min' for var in era5_daily_min.data_vars})
era5_daily_max = era5_daily_max.rename_vars({var: var + '_max' for var in era5_daily_max.data_vars})
era5_daily_sum = era5_daily_sum.rename_vars({var: var + '_sum' for var in era5_daily_sum.data_vars})
ds_era5_daily = xr.merge([era5_daily_mean, era5_daily_min, era5_daily_max, era5_daily_sum])

In [ ]:
enc = {x: {"compressor": Blosc()} for x in ds_era5_daily}

In [ ]:
ds_era5_daily.chunks

In [ ]:
ds_era5_daily.chunk({'time':-1, 'lat': -1, 'lon': -1}).to_zarr('//openmediavault/DataCold/SnowData/era5_snodas/era5_tmp_daily2.zarr', consolidated=True, encoding=enc)

In [ ]:
elevation = elevation_ds.rename({'x':'lon', 'y':'lat', 'band_data':'elevation'})

In [ ]:
ds_era5.time

In [ ]:
elevation = elevation.assign_coords(time=list(ds_era5.time.values))

In [ ]:
enc = {x: {"compressor": Blosc()} for x in elevation}
elevation.chunk({'lat': -1, 'lon': -1}).to_zarr('//openmediavault/DataCold/SnowData/era5_snodas/elevation.zarr', consolidated=True, encoding=enc)

In [10]:
xr.open_dataset('F:\\OAPMLData\\1.RawWeatherData\\gfs\\16\\US\\gfs.0p25.2016012500.f021.grib2.nc')

<xarray.Dataset>
Dimensions:                                      (latitude: 136,
                                                  longitude: 274, time: 1)
Coordinates:
  * latitude                                     (latitude) float64 24.25 ......
  * longitude                                    (longitude) float64 -130.5 ....
  * time                                         (time) datetime64[ns] 2016-0...
Data variables: (12/326)
    UGRD_localleveltype2200                      (time, latitude, longitude) float32 ...
    VGRD_localleveltype2200                      (time, latitude, longitude) float32 ...
    var0_2_224_localleveltype2200                (time, latitude, longitude) float32 ...
    GUST_surface                                 (time, latitude, longitude) float32 ...
    HGT_10mb                                     (time, latitude, longitude) float32 ...
    TMP_10mb                                     (time, latitude, longitude) float32 ...
    ...                                           ...
    TMP_PV_EQ_M2147D48_Km_2_kg_s_surface         (time, latitude, longitude) float32 ...
    HGT_PV_EQ_M2147D48_Km_2_kg_s_surface         (time, latitude, longitude) float32 ...
    PRES_PV_EQ_M2147D48_Km_2_kg_s_surface        (time, latitude, longitude) float32 ...
    var0_2_192_PV_EQ_M2147D48_Km_2_kg_s_surface  (time, latitude, longitude) float32 ...
    PRMSL_meansealevel                           (time, latitude, longitude) float32 ...
    var0_3_193_500mb                             (time, latitude, longitude) float32 ...
Attributes:
    Conventions:          COARDS
    History:              created by wgrib2
    GRIB2_grid_template:  0

In [12]:
## Rechunk the gfs data to align and be daily
gfs_ds = xr.open_mfdataset('F:/OAPMLData/1.RawWeatherData/gfs/*/US/*201*.nc', combine='nested', concat_dim='time', chunks={'time':4}, parallel=False)

In [ ]:
gfs_ds.chunks

In [11]:
list(gfs_ds.time.values)

[numpy.datetime64('2015-01-15T03:00:00.000000000'),
 numpy.datetime64('2015-01-15T06:00:00.000000000'),
 numpy.datetime64('2015-01-15T09:00:00.000000000'),
 numpy.datetime64('2015-01-15T12:00:00.000000000'),
 numpy.datetime64('2015-01-15T15:00:00.000000000'),
 numpy.datetime64('2015-01-15T18:00:00.000000000'),
 numpy.datetime64('2015-01-15T21:00:00.000000000'),
 numpy.datetime64('2015-01-16T00:00:00.000000000'),
 numpy.datetime64('2015-01-16T03:00:00.000000000'),
 numpy.datetime64('2015-01-16T06:00:00.000000000'),
 numpy.datetime64('2015-01-16T09:00:00.000000000'),
 numpy.datetime64('2015-01-16T12:00:00.000000000'),
 numpy.datetime64('2015-01-16T15:00:00.000000000'),
 numpy.datetime64('2015-01-16T18:00:00.000000000'),
 numpy.datetime64('2015-01-16T21:00:00.000000000'),
 numpy.datetime64('2015-01-17T00:00:00.000000000'),
 numpy.datetime64('2015-01-17T03:00:00.000000000'),
 numpy.datetime64('2015-01-17T06:00:00.000000000'),
 numpy.datetime64('2015-01-17T09:00:00.000000000'),
 numpy.datet

In [13]:
gfs_ds = gfs_ds.sortby('time')

In [ ]:

gfs_resample = gfs_ds.resample(time='1D')
gfs_daily_mean = gfs_resample.mean()
gfs_daily_min = gfs_resample.min()
gfs_daily_max = gfs_resample.max()
gfs_daily_sum = gfs_resample.sum()
gfs_daily_mean = gfs_daily_mean.rename_vars({var: var + '_mean' for var in gfs_daily_mean.data_vars})
gfs_daily_min = gfs_daily_min.rename_vars({var: var + '_min' for var in gfs_daily_min.data_vars})
gfs_daily_max = gfs_daily_max.rename_vars({var: var + '_max' for var in gfs_daily_max.data_vars})
gfs_daily_sum = gfs_daily_sum.rename_vars({var: var + '_sum' for var in gfs_daily_sum.data_vars})
ds_era5_daily = xr.merge([gfs_daily_mean, gfs_daily_min, gfs_daily_max, gfs_daily_sum])

2023-06-06 15:14:45,610 - distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)


# create a training sample
# for every era5 + elevation create a sample with target being snodas

In [ ]:
#lat = 40.77
#lon = -113.8

In [ ]:
era5 = xr.open_zarr('//openmediavault/DataCold/SnowData/era5_snodas/era5_tmp.zarr')
elevation = xr.open_zarr('//openmediavault/DataCold/SnowData/era5_snodas/elevation.zarr')
snow = xr.open_zarr('//openmediavault/DataCold/SnowData/snodas/snodas.zarr')